In [1]:
from src.run_exp import exp
import json


C:\Users\ctway\anaconda3\envs\distributed_GCN\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from src.traditional_solver import solve_maxsat_z3, solve_maxsat_FM, solve_uf_unknown
solve_uf_unknown('./data/sat_data/uuf50-test/uuf50-03.cnf')

UNSAT
Please using solve_maxcut_z3 or solve_maxcut_FM to get the maxcut result


In [2]:
solve_uf_unknown('./data/sat_data/uf50-test/uf50-01.cnf')

[46 = False,
 30 = False,
 13 = True,
 17 = False,
 36 = True,
 15 = False,
 7 = True,
 47 = True,
 41 = False,
 37 = False,
 6 = True,
 33 = False,
 34 = True,
 0 = False,
 8 = True,
 12 = False,
 49 = False,
 3 = True,
 38 = True,
 29 = False,
 9 = True,
 11 = True,
 1 = True,
 19 = True,
 24 = False,
 45 = False,
 31 = False,
 32 = False,
 10 = False,
 21 = False,
 39 = False,
 23 = False,
 35 = True,
 5 = True,
 40 = False,
 26 = True,
 44 = False,
 42 = False,
 2 = False,
 16 = False,
 28 = False,
 43 = False,
 14 = False,
 27 = False,
 22 = True,
 25 = False,
 48 = True,
 4 = True,
 20 = False,
 18 = True]

In [5]:
num_vio, model = solve_maxsat_FM('./data/sat_data/uf50-test/uf50-03.cnf')
num_vio


0

In [1]:
params = {
    "data": "stanford",
    "mode": "maxcut",
    "K": 1,

    "lr": 2e-3,
    "epoch": 1e3,
    "mapping": "distribution",
    "logging_path": "./log/maxcut_example.log",
    "res_path": "./res/maxcut_example.hdf5",
    "folder_path": "./data/maxcut_data/stanford_data/",
    "N_realize": 20, 
    "Niter_h": 120,
    "hyper": True,
    "penalty": 0,
}

In [6]:
from src.data_reading import read_uf, read_stanford, read_hypergraph
import numpy as np
import torch
import random

def loss_(probs, C):
    loss = 0
    for c in C:
        temp_1s = 1
        temp_0s = 1
        for index in c:
            temp_1s *= (1 - probs[index-1])
            temp_0s *= (probs[index-1])
        temp = (temp_1s + temp_0s - 1)
        loss += temp
    return loss
def mapping(best_outs, params, n, info):
    for rea in range(params['N_realize']):
        res = {x: np.random.choice(range(2), p=[1 - best_outs[x], best_outs[x]]) for x in best_outs.keys()}
        ord = random.sample(range(0, n), n)
        t = 1
        for it in range(params['Niter_h']):
            res1 = res
            for i in ord:
                temp = res.copy()
                # temp = pr.copy()
                if res[i] == 0:
                    temp[i] = 1
                else:
                    temp[i] = 0
                lt= loss_(temp, info[i+1])
                l1= loss_(res, info[i+1])
                if lt < l1 or np.exp(- (lt - l1) / t) > np.random.uniform(0, 1):
                    res = temp
            t = t * 0.95
    return res
test = './data/maxcut_data/stanford_data/G15.txt'
constraints, header = read_stanford(test)
n = header['num_nodes']
f = int(np.sqrt(n))
info = {x+1:[] for x in range(n)}
for constraint in constraints:
    for node in constraint:
        info[abs(node)].append(constraint) 

probs = torch.rand(n, requires_grad=True)

In [19]:
from src.data_reading import read_uf, read_stanford, read_hypergraph
test = './data/maxcut_data/stanford_data/G15.txt'
constraints, header = read_stanford(test)
len(constraints), header['num_nodes']

(4660, 800)

In [35]:
import random

In [ ]:
import random
def create_machines(constraints, n, n_machines):
    
    length = n // n_machines
    belonging = list(range(1, n+1))
    random.shuffle(belonging)
    belonging_group = [belonging[(i*length+1):((i+1)*length+1)] for i in range(n_machines)]
    belonging_dct = {}
    for i, group in zip(range(len(belonging_group)), belonging_group):
        for node in group:
            belonging_dct[node] = i
    info = {x+1:[] for x in range(n)}
    for constraint in constraints:
        for node in constraint:
            info[abs(node)].append(constraint) 
    info_machine = {x+1:[] for x in range(n)}

In [16]:
from utils import analysis_res
analysis_res('./res/maxcut_example_centralized.hdf5')

,File_name,Result
0,G1.txt,-11463
1,G10.txt,-11495
2,G11.txt,-1544
3,G12.txt,-1564
4,G13.txt,-1566
5,G14.txt,-3030
6,G15.txt,-3013
7,G2.txt,-11496
8,G3.txt,-11504
9,G4.txt,-11507


In [7]:
last_loss = 0
for i in range(int(params['epoch'])):
    probs_ = torch.sigmoid(probs)
    loss = loss_(probs_, constraints)
    loss.backward()
    if (last_loss - loss).item() <= 1e-2:
        last_loss = loss
        break
    if i % 100 == 0:
        print(f'at epoch {i}, loss = {loss.item()}')
    probs = probs.clone().detach() - probs.grad * params['lr']
    probs.requires_grad_()

at epoch 0, loss = -2181.626220703125


KeyboardInterrupt: 

In [8]:
probs = torch.sigmoid(probs)
probs = probs.detach().numpy()
probs = {i: probs[i] for i in range(len(probs))}
reses = mapping(probs, params, n, info)

In [11]:
reses = loss_(reses, constraints)

In [12]:
reses

-3004